In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import graphviz
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.metrics import top_k_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score

### Carregamento do conjunto de dados de todos os anos

In [2]:
ano = 2021
enade_ano = pd.read_csv(f"../../concept_drift/tabela_final_{ano}_treinamento.csv")

### Cross-validation (k = 5)

In [3]:
k = 5

In [4]:
array_folds = np.array_split(enade_ano, k)

In [5]:
for i_fold in range(k):
    
    # Separação dos folds
    
    folds_treinamento = array_folds.copy()
    fold_teste = array_folds[i_fold]
    
    del folds_treinamento[i_fold]
    
    folds_treinamento = pd.concat(folds_treinamento, sort=False)
    
    # Separação em X e y
    
    numero_caracteristicas = enade_ano.shape[1] - 1
    
    X_folds_treinamento = folds_treinamento.iloc[:, 0:numero_caracteristicas]
    y_folds_treinamento = folds_treinamento.iloc[:, -1]
    
    X_fold_teste = fold_teste.iloc[:, 0:numero_caracteristicas]
    y_fold_teste = fold_teste.iloc[:, -1]
    
    # Normalização de treino e teste
    
    normalizador_treinamento = StandardScaler()
    
    normalizador_treinamento.fit(X_folds_treinamento)
    treinamento_normalizado = normalizador_treinamento.transform(X_folds_treinamento)
    
    normalizador_teste = StandardScaler()
    
    normalizador_teste.fit(X_fold_teste)
    teste_normalizado = normalizador_teste.transform(X_fold_teste)
    
    # Busca por hyperparâmetros
    
    parametros_para_busca = {"n_estimators": [60, 100, 200],
                              "criterion":['gini', 'entropy'],
                              "min_samples_leaf": [10, 30, 50],
                              "ccp_alpha": [0.001, 0.0015, 0.0017, 0.002]}
    
    classificador = RandomForestClassifier()
    
    busca = HalvingGridSearchCV(classificador, parametros_para_busca, scoring="f1_weighted").fit(treinamento_normalizado, y_folds_treinamento)
    
    busca.best_estimator_.fit(treinamento_normalizado, y_folds_treinamento)
    y_predito = busca.best_estimator_.predict(teste_normalizado)
    
    melhor_classificador = str(busca.best_estimator_)
    resultado = f1_score(y_fold_teste, y_predito, average='weighted')
    
    print(f"Teste: fold {i_fold}")
    print(f"Classificador: \n\n{melhor_classificador}\n")
    print(f"Resultado: {resultado}\n")

Teste: fold 0
Classificador: 

RandomForestClassifier(ccp_alpha=0.001, criterion='entropy',
                       min_samples_leaf=10, n_estimators=60)

Resultado: 0.5015981624974305

Teste: fold 1
Classificador: 

RandomForestClassifier(ccp_alpha=0.0015, criterion='entropy',
                       min_samples_leaf=10, n_estimators=200)

Resultado: 0.45965273129363704

Teste: fold 2
Classificador: 

RandomForestClassifier(ccp_alpha=0.001, criterion='entropy',
                       min_samples_leaf=10)

Resultado: 0.4701135731094308

Teste: fold 3
Classificador: 

RandomForestClassifier(ccp_alpha=0.001, criterion='entropy',
                       min_samples_leaf=10)

Resultado: 0.4590075520915507

Teste: fold 4
Classificador: 

RandomForestClassifier(ccp_alpha=0.001, criterion='entropy',
                       min_samples_leaf=10)

Resultado: 0.5128348417863583



### Matriz de Confusão

In [14]:
y_pred = search.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=search.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=search.classes_)
disp.plot()
plt.show()